In [ ]:
from mfn import MFN3
import torch
from torch.utils.data import DataLoader
from mosei_dataset import MOSEIDataset
import pickle as pkl

In [2]:
data_path = "tensors.pkl"
device = 'cuda' if torch.cuda.is_available() else 'cpu'
test_dataloader = DataLoader(MOSEIDataset(data_path, "test"), batch_size=64, shuffle=False)

model = MFN3()
checkpoint = torch.load("workdir_MFN3_sentiment_lr0.0005_wd0.0_0223020207\epoch20_valloss0.01_valmae0.61.pt")
model.load_state_dict(checkpoint)
model.to(device)

TARGET_INDEX = {
    "sentiment": 0,
    "happy": 1,
    "sad": 2,
    "anger": 3,
    "surprise": 4,
    "disgust": 5,
    "fear": 6,
}

In [9]:
vid_keys = pkl.load(open("fold_to_vidkeys.pkl", "rb"))
vid_keys = vid_keys["test"]

In [14]:


model.eval()
batch_start_index = 0

for *inputs, labels in test_dataloader:
    inputs = [inputs[1], inputs[3], inputs[4]]
    inputs = torch.cat(inputs, dim=2).to(device, dtype=torch.float32)
    labels = labels[:, TARGET_INDEX["sentiment"]].to(device, dtype=torch.float32) # regression targets
    outputs = model(inputs)
    outputs = outputs.squeeze()

    maes = list(torch.abs(outputs - labels).cpu())
    for i, mae in enumerate(maes):
        if mae > 3:
            print(vid_keys[batch_start_index + i], end=" ")
            print(f"groud truth = {labels[i].item()}", end=" ")
            print(f"predicted = {outputs[i].item()}")


    batch_start_index += labels.shape[0]

111881[8] groud truth = -2.0 predicted = 1.0739003419876099
111881[10] groud truth = -2.0 predicted = 1.1930171251296997
112425[3] groud truth = 2.6666667461395264 predicted = -0.5479336977005005
112425[5] groud truth = 2.3333332538604736 predicted = -1.0635606050491333
198112[3] groud truth = 2.3333332538604736 predicted = -2.2885186672210693
208322[8] groud truth = -2.3333332538604736 predicted = 0.7543882131576538
238063[12] groud truth = 2.0 predicted = -1.116330623626709
24351[3] groud truth = 2.0 predicted = -1.5847101211547852
28006[0] groud truth = -2.6666667461395264 predicted = 0.5640744566917419
41381[10] groud truth = 1.0 predicted = -2.031898260116577
91844[7] groud truth = 2.6666667461395264 predicted = -0.8367092609405518
WoL4fCxGd8Q[6] groud truth = -2.3333332538604736 predicted = 0.7395752668380737
fsBzpr4k3rY[0] groud truth = 2.0 predicted = -1.2238091230392456
sfaWfZ2-4c0[1] groud truth = -2.3333332538604736 predicted = 1.1371482610702515


In [20]:
VID_KEY = "208322"

with open("vidkey_to_transcription.pkl", "rb") as f:
    key_to_text = pkl.load(f)

print(" ".join(key_to_text[VID_KEY]))

hi my name is mike and this is my review for mr wonder emporium and uhh this was a film that we took our two year old to uhh just a couple days ago umm it's probably one of the worst films uhh i've seen probably in the last five or six years umm the main reason why was there wasn't much of a plot but it was pretty boring uhh not not stutter really a lot of interesting things uhh seemed to occur during the uhh during the show umm one of the things that was uhh kind of interesting is it seems to be a cross between willy and uhh and some other christmas style movies uhh in which they're just hoping to kind of show you some uhh interesting colorful scenery uhh and have a toy store in which you walk by things and things pop out and weird stuff happens but uhh that just after maybe a minute or two of that and it becomes you become used to it you're just really not interested in seeing any more of that uhh dustin hoffman uhh is kind of bizarre uhh cast as this uhh particular person and natali